In [ ]:
from sctoolbox.utilities import bgcolor

# Embedding and clustering
<hr style="border:2px solid black"> </hr>

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
%bgcolor PowderBlue

#Number of threads to use for multiprocessing
threads = 4

# Number of PCs
n_pcs = None #this overwrites the automatic selection of PCs. Set to None to set number of PCs automatically

#Set number of neighbors
n_neighbors = 15 #Set the number of nearest neighbors to be used in clustering. Default=15

# Search umap parameters (or set parameters later)
search_umap_parameters = True

# Search different clustering resolutions
search_clustering_parameters = True
clustering_method = "leiden" #leiden/louvain

<hr style="border:2px solid black"> </hr>

## Loading packages

In [ ]:
import os
from os import path
import scanpy as sc
from kneed import KneeLocator
import matplotlib.pyplot as plt

import sctoolbox.utilities as utils
import sctoolbox.analyser as analyser
import sctoolbox.plotting as pl

## Load anndata from previous notebook

In [ ]:
adata = utils.load_anndata(is_from_previous_note=True, which_notebook=3)
display(adata)

In [ ]:
condition_column = adata.uns['infoprocess']['data_to_evaluate']

In [ ]:
figure_path = adata.uns["infoprocess"]["Anndata_path"] + "/"

## Subset number of PCs 
 Find initial neighbors and calculate differential expression
- NOTE: trim: Set to 0 to skip default 10. trims neighbours of cells can help to identify individual populations. Lower value more population but more batch
- NOTE: n_pcs: number of pca dimensions

In [ ]:
if n_pcs is None:
    n_pcs = analyser.define_PC(adata)

In [ ]:
ax = pl.plot_pca_variance(adata)
ax.axvline(n_pcs-0.5, color="red", label=f"n PCs included: {n_pcs}")
plt.legend()
utils.save_figure(figure_path + "PC_selection.pdf")

In [ ]:
# Subset PCA
analyser.subset_PCA(adata, n_pcs)

## Rerun neighbors

In [ ]:
sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs)

## Calculate UMAP and find best setting

- NOTE: min_dist: distances between points to make the plot looks more 'clustered'
- NOTE: spread: The effective scale of embedded points value be de default is 1

In [ ]:
if search_umap_parameters:
    pl.search_umap_parameters(adata, metacol=condition_column, threads=threads,
                              save=figure_path + "UMAP_parameter_search.pdf")

In [ ]:
%bgcolor PowderBlue

# Final choice of spread / dist
min_dist = 0.4
spread = 2.5

In [ ]:
# Plot final UMAP with quality measures
sc.tl.umap(adata, min_dist=min_dist, spread=spread)
sc.pl.umap(adata, color=condition_column)

In [ ]:
# Plot distribution of samples in umap
pl.plot_group_embeddings(adata, groupby=condition_column, save=figure_path + "sample_distribution_umap.png")

## Cell clustering
- NOTE: resolution: controls the coarseness of the clustering. Higher values lead to more clusters.

In [ ]:
if search_clustering_parameters:
    pl.search_clustering_parameters(adata, ncols=4, method=clustering_method, 
                                    save=figure_path + "clustering_search.png")

In [ ]:
%bgcolor PowderBlue

#Choose final resolution
clustering_column = "leiden_0.5"

### Reclustering

Here you can use the `analyser.recluser` function to iteratively adjust clustering

In [ ]:
analyser.recluster(adata, clustering_column, ["1", "3"], task="join")
clustering_column = "leiden_0.5_recluster"  #update clustering column

In [ ]:
analyser.recluster(adata, clustering_column, ["3"], task="split", resolution=0.15, key_added=clustering_column)  #overwrite column

In [ ]:
#Create final clustering
adata.obs["clustering"] = analyser.rename_categories(adata.obs[clustering_column])
adata.uns["infoprocess"]["plot_metrics"] = list(adata.uns["infoprocess"]["plot_metrics"]) + ["clustering"]

### Final clustering

In [ ]:
#Plot final leiden
sc.pl.umap(adata, color=[condition_column, "clustering"], show=False)
utils.save_figure(figure_path + "umap_final.pdf")

In [ ]:
# Plot final plot with QC metrics
sc.pl.umap(adata, color=adata.uns["infoprocess"]["plot_metrics"], ncols=3, show=False)
utils.save_figure(figure_path + "umap_qc.pdf")

## Plot distribution of cells across clusters

In [ ]:
pl.n_cells_barplot(adata, "clustering", groupby=condition_column, 
                   save=figure_path + "cell_distribution_barplot.pdf")

## Saving adata for next notebook

In [ ]:
utils.saving_anndata(adata, current_notebook=4)